# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [6]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [7]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [22]:
driver_path = r'D:\Documents\Jupyter_dir\chromedriver_win32_85\chromedriver.exe'
browser = webdriver.Chrome(driver_path)
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新

# 提示：可參考以下的Stack Overflow: 
# https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    print("網頁更新中...")
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    
    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        print("到達頁面底端")
        break
    last_height = new_height

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [23]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            return p.string

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [24]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

news = ""
for i, news_item in enumerate(news_block):
    print("Get news: ", i)
    news += "----------------------------------------------------------------------\n"
    news_body = news_item.find('h3')
    news += "\n[%d] %s\n" % (i, news_body.a.string)
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    try:
        news += getNewsDetailContent(externalLink) + "\n"
    
    except Exception:
        pass

print("All done")

Get news:  0
Get news:  1
Get news:  2
Get news:  3
Get news:  4
Get news:  5
Get news:  6
Get news:  7
Get news:  8
Get news:  9
Get news:  10
Get news:  11
Get news:  12
Get news:  13
Get news:  14
Get news:  15
Get news:  16
Get news:  17
Get news:  18
Get news:  19
Get news:  20
Get news:  21
Get news:  22
Get news:  23
Get news:  24
Get news:  25
Get news:  26
Get news:  27
Get news:  28
Get news:  29
Get news:  30
Get news:  31
Get news:  32
Get news:  33
Get news:  34
Get news:  35
Get news:  36
Get news:  37
Get news:  38
Get news:  39
Get news:  40
Get news:  41
Get news:  42
Get news:  43
Get news:  44
Get news:  45
Get news:  46
Get news:  47
Get news:  48
Get news:  49
Get news:  50
Get news:  51
Get news:  52
Get news:  53
Get news:  54
Get news:  55
Get news:  56
Get news:  57
Get news:  58
Get news:  59
Get news:  60
Get news:  61
Get news:  62
Get news:  63
Get news:  64
Get news:  65
Get news:  66
Get news:  67
Get news:  68
Get news:  69
Get news:  70
Get news:  71
Ge

In [25]:
news

'----------------------------------------------------------------------\n\n[0] 郭台銘赴美為新冠肺炎唾液快篩機宣傳？\u3000劉宥彤：僅雛形階段\n▲郭台銘從中秋節便赴往美國。（圖／記者湯興漢攝）\n----------------------------------------------------------------------\n\n[1] 台積電製程不斷推進 牽動科學園區發展\n▲台積電加持下南科將邁入高成長期。（圖／達志影像／美聯社）\n----------------------------------------------------------------------\n\n[2] 年薪破百萬！九職等老爸仍要求「考公職」…網點「1關鍵」狂勸退\n▲許多人努力考公職，希望可以取得穩定的工作。（示意圖／取自免費圖庫pxhere）\n----------------------------------------------------------------------\n\n[3] 史上最嚴峻水情！水利署於翡翠水庫啟動增雨「催化降雨雲系」\n▲經濟部水利署昨(16)日晚間於翡翠水庫集水區東側上游的碧湖站完成增雨作業。（圖／經濟部提供）\n----------------------------------------------------------------------\n\n[4] 豪宅回春訊號！\u3000國產署都更宅熱呼呼\u3000「明日博」熱銷\n▲位在台北市大安區市民大道三段「明日博」熱銷，外界視為豪宅市場回溫指標。（圖／記者宋良義攝）\n----------------------------------------------------------------------\n\n[5] 南韓房地產失控飆！文在寅「打房21次」全失敗\u3000律師月薪20萬買不起\n▲南韓祭出第22次打房政策抑制房價。（圖／達志影像）\n----------------------------------------------------------------------\n\n[6] 科技股尾盤湧現賣壓\u3000美股道瓊漲112點\n▲美股漲跌互見，科